# Clustering

In this course we've already learned about classification, which assumes we have a test set of data that we have already categorized. When we run models with labeled data, we call it _supervised learning_.

But what happens when we don't have labeled data? Is it possible to learn anything about what the data contains?

This scenario is known as *unsupervised learning*, and we will see that it is very possible to learn about the underlying structure of unlabeled observations.

## An example

Here’s a scenario:

Suppose you scrape thousands of news articles. And suppose you want to organize, or cluster, those documents by their prevalence of language about hurricanes vs elections. You could choose a group of words relate to hurricanes and make the x-axis the frequency with which those words appear in the documents. You could do the same for words related to elections on the y-axis. We might imagine the articles clustering into four groups: one for documents that are largely about a hurricane, another for documents largely about a election, a third for documents about neither, and a fourth for documents, however unlikely, about both.

These clusters represent an underlying structure of the data. But it's still supervised, because we've labeled words as belonging to categories and used them to organize documents.

Unsupervised learning applies the same basic idea, but in a high-dimensional space with one dimension for every word. As such, the space can’t be visualized. But the goal is the same as in two-dimensions: identify an underlying structure of the observed data, such that documents cluster, and each cluster is internally coherent.

**Clustering algorithms** are capable of finding such structure automatically.

## Enter k-means clustering

Clustering algorithms assign each data point—for us, each document—to a discrete cluster. One of the best known clustering algorithms is k-means, an iterative algorithm that maintains a cluster assignment for each instance, and a central ("mean") location for each cluster. K-means iterates between updates to the assignments and the centers:

1. each instance is placed in the cluster with the closest center;

2. each center is recomputed as the average over points in the cluster.

Let's watch [the start of a little video](https://www.youtube.com/watch?v=yR7k19YBqiw) to see how it works.

**NOTE:** An important property of k-means is that the converged solution depends on the initialization, and a better clustering can sometimes be found simply by re-running the algorithm from a different random starting point.

## Let's get to it! ##

In this example, we're going to use k-means clustering to identify the latent structures within the synopses of the top 100 films of all time (per an IMDB list), a corpus created by Brandon Rose. See [the original post](http://www.brandonrose.org/top100) for a more detailed discussion on the corpus.

Let's start with the imports. They include old friends: re, BeautifulSoup, nltk, pandas, sklearn, a veritable journey through our semester:

In [ ]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
from sklearn import feature_extraction

## Corpus

Now let's load in our corpus. Here, we're going to import four files that each map to each other: film titles, links to the films on imdb, wikipedia synopses of the films, and the associated genres for each.


In [ ]:
# For downloading large files from Google Drive
import gdown

# Download the title list file - a list of film titles, in order of top-ness
gdown.download('https://drive.google.com/uc?export=download&id=15FqA2-VKSoGdT3DT2IBiSCeIROQup2LH', quiet=False)

# Download the url list - a list of URLS to each film on IMDB, in the same order as the first
gdown.download('https://drive.google.com/uc?export=download&id=1LMbHmHlAZXWJN0KAIxbvfy5zB_6sAwio', quiet=False)

# Download the synopses file - a list of synopses of each film, in the same order as both files above, with "BREAKS HERE" separating each entry
gdown.download('https://drive.google.com/uc?export=download&id=1BDRU4wcG6Qd1zaQwyfQouufw2hmAEOKr', quiet=False)

# Download the genres file - a list of associated genres, one film per line, in the same order as above
gdown.download('https://drive.google.com/uc?export=download&id=1Phf-rcmQXneEiUra5z_l5Eky7x6FDITk', quiet=False)


## Corpus Pre-Proccesing ##

Now let's pre-process our corpus for clustering.

In [ ]:
# import four files that each map to each other: titles, links, wikipedia synopses, and associated dgenres

# first, the list of film titles, in order of top-ness
titles = open('title_list.txt').read().split('\n')

# ensures that only the top 100 titles are read in
titles = titles[:100]

# a list of URLS to each film on IMDB, in the same order as the first
links = open('link_list_imdb.txt').read().split('\n')
links = links[:100]

# a list of synopses of each film, in the same order as both files above, with "BREAKS HERE" separating each entry
synopses_wiki = open('synopses_list_wiki.txt').read().split('\n BREAKS HERE')
synopses_wiki = synopses_wiki[:100]

synopses_clean_wiki = []

for text in synopses_wiki:
    text = BeautifulSoup(text, 'html.parser').getText()
    # strips html formatting and converts to unicode
    synopses_clean_wiki.append(text)

synopses_wiki = synopses_clean_wiki

# now the list of associated genres, one film per line, in the same order as above
genres = open('genres_list.txt').read().split('\n')
genres = genres[:100]

print(str(len(titles)) + ' titles')
print(str(len(links)) + ' links')
print(str(len(synopses_wiki)) + ' synopses')
print(str(len(genres)) + ' genres')

Let's see what movies we have and their genres:

In [ ]:
for x in range(99):
    print(titles[x], genres[x])

Let's check out the first synopsis

In [ ]:
print(synopses_wiki[0])

In [ ]:
# now let's get the imdb synopses as well
gdown.download('https://drive.google.com/uc?export=download&id=1NofscgV8eWPhgM4QxSAN3cyIhwENqIk4', quiet=False)

synopses_imdb = open('synopses_list_imdb.txt').read().split('\n BREAKS HERE')
synopses_imdb = synopses_imdb[:100]

synopses_clean_imdb = []

for text in synopses_imdb:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    synopses_clean_imdb.append(text)

synopses_imdb = synopses_clean_imdb

# and check out the first
synopses_imdb[0]

In [ ]:
# make a list with the two sets of synopses—from imdb and wikipedia

synopses = []

for i in range(len(synopses_wiki)):
    item = synopses_wiki[i] + synopses_imdb[i]
    synopses.append(item)

# see what one looks like
print(synopses[0])

## Now our trusty friend tf-idf

Before we can cluster our movie synopses, we need to prepare them as vectors. We're going to prepare them as *tf-idf* vectors because they provide more effective clusters than other ways of counting words, for example raw word counts.

NB: Some people like to lemmatize or stem their documents before peparing the vectors, which we also now know how to do!

In [ ]:
#import spacy
#import en_core_web_sm

# load the  model
#nlp = spacy.load("en_core_web_sm")

# lemmatize the synopses
#lemmatized_synopses = []

#for synopsis in synopses:
#    doc = nlp(synopsis)
#    lemmatized_synopsis = " ".join([token.lemma_ for token in doc])
#    lemmatized_synopses.append(lemmatized_synopsis)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', #note new params
                                   use_idf=True,
                                    ngram_range=(1,3), # use n-grams from 1 to 3 words long
                                   min_df=.2, # filter out n-grams that appear in less than 20% of documents
                                   max_df=0.8) # ignore n-grams that appear in more than 80% of documents

# tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(lemmatized_synopses)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(synopses)

print(tfidf_vectorizer_vectors.shape)

**What does the shape of the vector tell us?**

In [ ]:
# get our feature names for future reference
# terms = tfidf_vectorizer.get_feature_names()
terms = tfidf_vectorizer.get_feature_names_out()
terms

In [ ]:
# get the first vector out (for the first synopsis) just to see what it looks like
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]

# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=terms, columns=["tfidf"])

df.sort_values(by=["tfidf"],ascending=False).head(10)

This is the vector for **The Godfather**. It scores high on 'don', 'family', and 'son'. How about we take a look at another one?

In [ ]:
# get the first vector out (for the first synopsis) just to see what it looks like
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[96]

# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=terms, columns=["tfidf"])

df.sort_values(by=["tfidf"],ascending=False).head(10)

**Any guesses as to what this one is?**

## On to the clustering!

Using our tf-idf vectors, we can now run the k-means clustering algorithm. Remember that k-means initializes with a pre-determined number of clusters. Let's somewhat arbitrarily choose 5.

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters, n_init=10) # default is also 10, but good to know

clusters = km.fit_predict(tfidf_vectorizer_vectors)

clusters

# Visualize the clusters

We can dimensionality-reduce our tf-idf vectors and plot them in order to see whether the clusters roughly align with the data, like so:

In [ ]:
import matplotlib.pyplot as plt

# recall that each film synopsis is a tf_idf vector in the list of tfidf_vectorizer_vectors

# now convert tfidf_vectorizer_vectors to a dense array for plotting
tfidf_vectorizer_vectors_dense = tfidf_vectorizer_vectors.toarray()

# reduce dimensionality for visualization using PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
tfidf_2d = pca.fit_transform(tfidf_vectorizer_vectors_dense)

# Create the scatter plot
plt.figure(figsize=(10, 7))
plt.scatter(tfidf_2d[:, 0], tfidf_2d[:, 1], c=clusters)
plt.title('Film synopses clustered by k-means')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# create a custom legend
legend_elements = []
for i in range(num_clusters):
  legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label=f'Cluster {i}', markerfacecolor=plt.cm.get_cmap('viridis', num_clusters)(i), markersize=10))

plt.legend(handles=legend_elements, title="Clusters")
plt.show()

Hm... At least looking at the first two components, these films don't seem very cluster-able, at least on the basis of their synopses. Let's just inspect the fourth cluster since that one seems the most distinct of the bunch.  

# Inspecting clusters

First we should dump our clusters into a dataframe for better inspection and manipulation

In [ ]:
films = { 'title': titles, 'idx': list(range(100)), 'synopsis': synopses, 'cluster': clusters, 'genre': genres }

film_df = pd.DataFrame(films, columns = ['title', 'idx', 'cluster', 'genre'])

film_df

Now we can inspect the clusters a bit more easily

In [ ]:
import textwrap

# find all films in each cluster
for i in range(num_clusters):
    print("Titles in cluster " + str(i) + ": ")
    cluster_titles = ""

    # create new df of only the specific cluster
    # remember boolean selection!
    cluster_df = film_df[ film_df["cluster"] == i ]

    # create series of titles assoc w/ that cluster
    for title in cluster_df['title']:
        cluster_titles += title + ", "

    print(textwrap.fill(cluster_titles, 100) + "\n")


To get some more insight, we could visualize the clusters by genre rather than by cluster. Let's do that now.

First, we need to add our PCA components to our dataframe:

In [ ]:
# add tfidf_2d to the dataframe in columns 'pca1' and 'pca2'
film_df['pca1'] = tfidf_2d[:, 0]
film_df['pca2'] = tfidf_2d[:, 1]

film_df

But we still need to clean up the genres a bit more.

Regex to the rescue!



In [ ]:
import re

cleaned_genres = []

for genre_string in film_df['genre']:
  cleaned_genre_string = re.sub(r"u'", '', genre_string) # clean the u' from the start of each genre
  cleaner_genre_string = re.sub("[\[\]\s\']+", "", cleaned_genre_string) # remove [, ], ', and whitespace
  cleaner_genre_list = cleaner_genre_string.split(',') # split into a list
  cleaned_genres.append(cleaner_genre_list)

# reassign to the genre column
film_df['genre'] = cleaned_genres

# check it out
film_df

Now we can plot the films according to genre

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Create a dictionary to map genres to colors
unique_genres = set()
for genres_list in film_df['genre']:
    for genre in genres_list:
        unique_genres.add(genre)

# Generate a colormap with a unique color for each genre
genre_colors = dict(zip(unique_genres, plt.cm.get_cmap('tab20', len(unique_genres))(range(len(unique_genres)))))

# Function to blend colors based on multiple genres
def blend_colors(genres_list):
    if len(genres_list) == 0 :
      return (0,0,0,0) # handle cases where a movie has no listed genres.

    r, g, b, a = 0, 0, 0, 0
    for genre in genres_list:
        if genre in genre_colors:
          cr, cg, cb, ca = genre_colors[genre]
          r += cr
          g += cg
          b += cb
          a += ca
        else:
          r += 0
          g += 0
          b += 0
          a += 0

    return (r / len(genres_list), g / len(genres_list), b / len(genres_list), a / len(genres_list))

# Create a list of blended colors for each movie
blended_colors = [blend_colors(genres_list) for genres_list in film_df['genre']]

# Plot PCA1 and PCA2 with blended colors
plt.figure(figsize=(10, 7))
plt.scatter(film_df['pca1'], film_df['pca2'], c=blended_colors)

plt.title('Film Synopses Clustered by k-means (Colored by Genre)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Create the custom legend
legend_elements = []
for genre, color in genre_colors.items():
    legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label=genre, markerfacecolor=color, markersize=10))

plt.legend(handles=legend_elements, title="Genres", bbox_to_anchor=(1.05, 1), loc='upper left')  # Adjust legend position

plt.show()

Same thing as above, just with boundary lines to see the clusters

In [ ]:
# prompt: modify the code above to also draw a boundary line around each cluster

# ... (Your existing code)

# Assuming film_df is already created as in your provided code

# Create a dictionary to map genres to colors
unique_genres = set()
for genres_list in film_df['genre']:
    for genre in genres_list:
        unique_genres.add(genre)

# Generate a colormap with a unique color for each genre
genre_colors = dict(zip(unique_genres, plt.cm.get_cmap('tab20', len(unique_genres))(range(len(unique_genres)))))

# Function to blend colors based on multiple genres
def blend_colors(genres_list):
    if len(genres_list) == 0 :
      return (0,0,0,0) # handle cases where a movie has no listed genres.

    r, g, b, a = 0, 0, 0, 0
    for genre in genres_list:
        if genre in genre_colors:
          cr, cg, cb, ca = genre_colors[genre]
          r += cr
          g += cg
          b += cb
          a += ca
        else:
          r += 0
          g += 0
          b += 0
          a += 0

    return (r / len(genres_list), g / len(genres_list), b / len(genres_list), a / len(genres_list))

# Create a list of blended colors for each movie
blended_colors = [blend_colors(genres_list) for genres_list in film_df['genre']]


# Plot PCA1 and PCA2 with blended colors and cluster boundaries
plt.figure(figsize=(10, 7))

# Plot data points
plt.scatter(film_df['pca1'], film_df['pca2'], c=blended_colors)

# new part!
# Calculate cluster boundaries using convex hull
from scipy.spatial import ConvexHull

for cluster_id in range(num_clusters):
    cluster_points = tfidf_2d[clusters == cluster_id]
    hull = ConvexHull(cluster_points)
    for simplex in hull.simplices:
        plt.plot(cluster_points[simplex, 0], cluster_points[simplex, 1], 'k-')


plt.title('Film Synopses Clustered by k-means (Colored by Genre, Boundaries)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Create the custom legend
legend_elements = []
for genre, color in genre_colors.items():
    legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label=genre, markerfacecolor=color, markersize=10))

plt.legend(handles=legend_elements, title="Genres", bbox_to_anchor=(1.05, 1), loc='upper left')  # Adjust legend position

plt.show()

OK. Still not convincing. On to evaluation.

## Evaluation

## Selecting K
So, the question remains: is this the right number for k?

One of the metrics commonly used to compare results across different values of k is called "inertia," which is a measure of within-cluster variance.

More specifically, inertia measures the sum of the squared distances between each datapoint and the nearest cluster center (or "centroid").

Lower inertia values indicate that points are closer to their assigned cluster centers, implying more compact clusters.

However, since increasing the number of clusters will always decrease the inertia, since the more clusters you have the closer the points can be to its center, inertia cannot be used as a standalone measure.

Instead, mean distance to the centroid as a function of k is plotted and the "elbow point," where the rate of decrease sharply shifts, can be used to roughly determine the ideal value for k.

Some other techniques exist for validating k, but that's the most common one.

Let's try it!

In [ ]:
inertia_vals = []

for i in range(1, 21):
    km_clustering = KMeans(n_clusters=i, n_init=10)
    km_clustering.fit(tfidf_vectorizer_vectors)
    inertia_vals.append(km_clustering.inertia_)

ks = list(range(1, 21))


In [ ]:
import matplotlib.pyplot as plt

plt.xlabel('number of clusters k')
plt.ylabel('Inertia')
plt.plot(ks, inertia_vals, 'bx-')


In [ ]:
# library for quickly calculating "knee"
%pip install kneed

In [ ]:
from kneed import KneeLocator
kn = KneeLocator(ks, inertia_list, curve='convex', direction='decreasing')
print(kn.knee)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('number of clusters k')
plt.ylabel('Inertia')
plt.plot(ks, inertia_list, 'bx-')
plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')

# A few miscellaneous things...

Finding top terms per cluster

In [ ]:
# find the top terms per cluster

# this orders by the distance of each term from the center
# (cluster_centers_ returns an array of [n_clusters, n_features] )
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(num_clusters):
    print("Cluster " + str(i) + " top words: ")
    top_terms = ""

    for ind in order_centroids[i, :10]:
        top_terms += terms[ind] + ", "

    print(top_terms)

Find top 10 films in each cluster

In [ ]:
# Find the top 10 films in each cluster
for i in range(num_clusters):

    # returns array w/ distances to the centroid i
    dist = km.transform(tfidf_vectorizer_vectors)[:, i]

    # return indices for top 10 for that cluster
    idxs = np.argsort(dist)[::][:10]

    print("Top 10 films in cluster " + str(i) + ": ")
    cluster_top_films = ""

    # look up film title by index
    for idx in idxs:
        cluster_top_films += film_df.loc[idx,'title'] + ", "

    print(cluster_top_films + "\n")

*Based on materials by Brandon Rose, Jacob Eisenstein, and Eun Seo Jo et al.*
